In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import other libraries as you need them here.
#### You will need to re-run all or just this one cell if you add more imports later. You may also ignore this and import as you need later.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
 
from sklearn.model_selection import train_test_split
 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

#### (1)
Load your DF. Display the first 5 rows.

In [ ]:
df = pd.read_csv('/kaggle/input/diamonds/diamonds.csv')
df.head(5)

#### (2)
Display the column names only.

In [ ]:
df.columns

#### (3)

Data clean up time.

First, let's re-index all of the columns so that price becomes the final column. While doing so, we will remove the excess column by not placing in the list to replace.

Display the df.

In [ ]:
df = df[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'price']]
df

#### (4)

Let's take a look at the current data graphically. This will be useful later when we want to clean up some potential misleading data.  

Seaborn is a library designed for some amazing data visualizations.  
Import: seaborn as sns  
Then use: sns.pairplot( your data frame, hue='cut', plot_kws={'s':1} )  
This will take a bit of time to run. You can try a different column for the hue, color or clarity should work well. The plot_kws is to make the size of the individual plots be smaller.  
After running it and studying it you may want to open the image in a new tab then comment this line out while you work on the rest of the project so you do not run it again. Uncomment it before submitting.


In [ ]:
# sns.pairplot(df, hue='cut', plot_kws={'s':1})


#### (5)

Some early project questions:

1) Which two potential features (variables) may not be useful for predicting price?

2) Which potential features look like they may be most useful for predicing price?

3) Can you identify some potential incorrect data that needs to be removed? Name a few features and some ranges we may want to remove. (example: remove data with age > 100 )


Depth and Table vs price all have a fairly verticle graph. This means that they wouldn't be able to accuratly predict the price of a diamond.

Carat and X both look like they could be used to predict an accurate price.

Any time x, y, or z = 0 should be removed.

#### (6)

Display all of the description statistics of the df.

In [ ]:
df.describe()

#### (7)

You should notice that some of the columns are missing in the statistics and were also missing in the PairGrid. This is because these are categorical data instead of numerical. Most machine learning algorithms will not be able to work with categorical data without converting the data first into numerical values. There are libraries which can convert categories to numbers automatically for you, but they will just assign a number to each category with no regards to the 'value' of one category over another if the categories could be sorted. For instance, Bristow may get a 0, Gainesville may get a 1, Haymarket may get a 2. The order here could be arbitrary or alphabetic and it would not matter.  

Some categorical values could be ordered. In diamonds, there is an order to the cut, color, and clarity values. One is 'better' than another.  

Convert your categorical data into numerical data.  
Fair < Good < Very Good < Premium < Ideal  
J < I < H < ... < D  
I1 < SI2 < SI1 < VS2 < VS1 < VVS2 < VVS1 < IF < FL

Display your DF to check the categories converted.

In [ ]:
df.color = df.color.replace(to_replace = ['J', 'I', 'H', 'G', 'F', 'E', 'D'], value=[0, 1, 2, 3, 4, 5, 6])
df.cut = df.cut.replace(to_replace = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'], value=[0, 1, 2, 3, 4])
df.clarity = df.clarity.replace(to_replace = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF', 'FL'], value=[0, 1, 2, 3, 4, 5, 6, 7, 8])
df


#### (8)

Let's deal with some of the outliers you may have noticed now. These may be explainable, but we are going to assume they are data that will interfere with our predictions. You should notice a few points in the y and z features that are definitely not consistent. There are also diamonds showing x/y/z values as 0, which present incomplete data we cannot use. Figure out a way to remove a good portion of these pieces of data that will interfere with our predictions.

Re-display all the main statistics of your DF.



In [ ]:
df = df.drop(df[(df.x == 0) | (df.y == 0) | (df.z == 0)].index)
df.describe()

#### (9)
Display the correlations of your DF, sorted in increasing order. You can either display all of them or limit the display only to the price column, which is the only correlation that will ultimately matter for our predictions.

In [ ]:
df.corr().price

#### (10)
Diamonds are often talked about with their "Four C's". Cut, Color, Clarity, and Carat. According to these correlations, rank the "Four C's" from most important to least important in price. (Think for a moment on this one!)  
  
Most Important  
1)  
2)  
3)  
4)  
Least Important  

Carrot, Color, clarity, and then Cut.

#### (11)
Now we will look a bit more at those correlations as a whole.  

A heatmap can be used to display the different correlations in a graphical way. A correlation can be anywhere from -1 to 1. A correlation of 0 means there is almost nothing connecting one variable to another. Closer to 1 means that it is almost guaranteed that as one variable goes up, another goes up almost linearly the same. Closer to -1 means as one goes up, another goes down almost linearly.

To display a heatmap, use: sns.heatmap( your dataframe's correlation ) or sns.heatmap( your dataframe's correlations, annot=True). Try out both to see the difference.

In [ ]:
sns.heatmap( df.corr(), annot = True ) 

#### (12)
These are similar questions to the earlier ones.

1) Which features seem to be the best to be used to predict the price of a diamond?  

2) Which features may need to be thrown out during predictions?  

3) There are several variables that are correlated so highly that perhaps it would be okay to remove some of them to cut down on computing times. Which variable(s) may you be able to cut and which variable(s) would you keep because it covers the same overall trends?

Carat, x, y, and z all seem to have high correlation values. 
Cut, color, clarity, depth, and table all have relativilty low correlation vlaues. 
Carat and price are very closely related, it may benefit from removing them.

#### (13)
Build a linear regression model.
Choose all of the features (aside from price) for your X values.
Choose price for your y values.

Split your data into two equal halves for training and validating.
Run your model on your training set.
Predict your y values based on your validation X values.

What is your mean absolute error?  


In [ ]:
model = LinearRegression()
X = df[['x', 'y', 'z', 'carat', 'cut', 'color', 'clarity', 'depth', 'table']]
Y = df.price
train_X, test_X, train_Y, test_Y, = train_test_split(X, Y, test_size = 0.5)
model.fit(train_X, train_Y)
predictions = model.predict( test_X )

mean_absolute_error(test_Y, predictions)

#### (14)
Display the coefficients your linear model computed in a nice chart showing the variable name and the computed coefficient.



In [ ]:
model.coef_

#### (15)

Hard-code a line of code using each of your found coefficients to calculate/display the predicted price of the following diamond:

Carat: 0.8  
Cut: Very Good  
Color: F  
Clarity: VVS2   
Depth: 61.8  
Table: 57.5  
x: 5.7  
y: 5.7  
z: 3.5  

In [ ]:
x_pred = 5.7 * -842.778
y_pred = 5.7 * 25.769
z_pred = 3.5 * -50.734
carat_pred = 0.8 * 10729.770
cut_pred = 2 * 124.074
color_pred = 4 * 327.155
clarity_pred = 5 * 500.737
depth_pred = 61.8 * -74.869
table_pred = 57.5 * -26.585
price = x_pred + y_pred + z_pred + carat_pred + cut_pred + color_pred + clarity_pred + depth_pred + table_pred
price + model.intercept_

#### (16)
Run another LinearRegression model, but this time use only the "Four C's" as your features. Display your mean absolute error.

In [ ]:
model = LinearRegression()
X = df[['carat', 'cut', 'color', 'clarity']]
Y = df.price
train_X, test_X, train_Y, test_Y, = train_test_split(X, Y, test_size = 0.5)
model.fit(train_X, train_Y)
predictions = model.predict( test_X )

mean_absolute_error(test_Y, predictions)

#### (17)
Try again, this time only with carat.

In [ ]:
model = LinearRegression()
X = df[['carat']]
Y = df.price
train_X, test_X, train_Y, test_Y, = train_test_split(X, Y, test_size = 0.5)
model.fit(train_X, train_Y)
predictions = model.predict( test_X )

mean_absolute_error(test_Y, predictions)

#### (18)
One more time with carat, x, y, and z (the four highest correlating features compared to price).

In [ ]:
model = LinearRegression()
X = df[['x', 'y', 'z', 'carat']]
Y = df.price
train_X, test_X, train_Y, test_Y, = train_test_split(X, Y, test_size = 0.5)
model.fit(train_X, train_Y)
predictions = model.predict( test_X )

mean_absolute_error(test_Y, predictions)

#### (19)
Compare the last three models used which utilized less features than the first.

1) Is there anything that seems non-intuitive about which of these three sets is best?

2) Can you attempt to explain why it is the best of these three?

3) Using all possible features gave the best result (hopefully). Assuming you had no concern over how long your model took to execute, can you think of a reason why you may want to leave a feature out of your model?


It is weird that the set with the best model used depth and table which had very little corelation.

Maybe it's simply a factor of the more data, the better the model.

If a feature was very random and had no correlation what so ever, it make be helpful to remove.

#### (20)
Create a copy of your dataframe, then add a new column 'predictions' and fill it with the results from predicting your model on the entire original X with all of the columns. (If you re-used the same variable for the last few regression models again, make sure you reset the model back to using all of the columns).  

Create another new column called abs_error. Set this column to equal the absolute value of the price column minus the predictions column to show how far off the prediction was.

Display your DF.

In [ ]:
model = LinearRegression()
X = df[['x', 'y', 'z', 'carat', 'cut', 'color', 'clarity', 'depth', 'table']]
Y = df.price
train_X, test_X, train_Y, test_Y, = train_test_split(X, Y, test_size = 0.5)
model.fit(train_X, train_Y)
predictions = model.predict( test_X )

mean_absolute_error(test_Y, predictions)

df2 = df
df2['predictions'] = df2.x * model.coef_[0] + df2.y * model.coef_[1] + df2.z * model.coef_[2] + df2.carat * model.coef_[3] + df2.cut * model.coef_[4] + df2.color * model.coef_[5] + df2.clarity * model.coef_[6] + df2.depth * model.coef_[7] + df2.table * model.coef_[8] + model.intercept_
df2['abs_error'] = abs(df2.price - df2.predictions)
df2

#### (21)
Display the full statistics on your DF again. Look particularly at the mean on the abs_error column. Does it match the number calculated above or is it slightly off? If it is off, can you explain why that may be?

In [ ]:
df2.describe()

It is fairly close but not exact. This is because I am testing with the whole table, where as that value was calculated using just the testing table.

#### (22)
Display a scatter plot comparing your price column to your predictions column. Ideally this would be an almost perfect line. You may want to make the size of the individual plotted points to be smaller. Title your plot with the type of regression used (Linear Regression in this case).

In [ ]:
df2.plot.scatter(x = 'price', y = 'predictions', s = 0.1, title = 'LinearRegression')

#### (23)
Now run the model as a decision tree using all variables. Again show the mean absolute error.

In [ ]:
model = DecisionTreeRegressor()
X = df[['x', 'y', 'z', 'carat', 'cut', 'color', 'clarity', 'depth', 'table']]
Y = df.price
train_X, test_X, train_Y, test_Y, = train_test_split(X, Y, test_size = 0.5)
model.fit(train_X, train_Y)
predictions = model.predict( test_X )

mean_absolute_error(test_Y, predictions)

#### (24)
Display a scatter plot comparing your price column to your predictions column. Ideally this would be an almost perfect line. You may want to make the size of the individual plotted points to be smaller. Title your plot with the type of regression used.

In [ ]:
df2.predictions = model.predict(df2[['x', 'y', 'z', 'carat', 'cut', 'color', 'clarity', 'depth', 'table']])
df2.plot.scatter(x = 'price', y = 'predictions', s = 0.1, title = 'Test')

#### (25)
Now run the model as a random forest.

In [ ]:
model = RandomForestRegressor()
X = df[['x', 'y', 'z', 'carat', 'cut', 'color', 'clarity', 'depth', 'table']]
Y = df.price
train_X, test_X, train_Y, test_Y, = train_test_split(X, Y, test_size = 0.5)
model.fit(train_X, train_Y)
predictions = model.predict( test_X )

mean_absolute_error(test_Y, predictions)

#### (26)
Display a scatter plot comparing your price column to your predictions column. Ideally this would be an almost perfect line. You may want to make the size of the individual plotted points to be smaller. Title your plot with the type of regression used.


In [ ]:
df2.predictions = model.predict(df2[['x', 'y', 'z', 'carat', 'cut', 'color', 'clarity', 'depth', 'table']])
df2.plot.scatter(x = 'price', y = 'predictions', s = 0.1, title = 'Test')

#### (27)
Choose the model with the least mean absolute error. Update the prediction column and the abs_error column to use this model's predictions.
Display your final df.

In [ ]:
model = RandomForestRegressor()
X = df[['x', 'y', 'z', 'carat', 'cut', 'color', 'clarity', 'depth', 'table']]
Y = df.price
train_X, test_X, train_Y, test_Y, = train_test_split(X, Y, test_size = 0.5)
model.fit(train_X, train_Y)
predictions = model.predict( test_X )

mean_absolute_error(test_Y, predictions)
df2['abs_error'] = abs(df2.price - df2.predictions)
df2